### Testing specific issues with the CV's
Some testing... Sara's failed due to the "ć"

In [7]:
import pdfplumber
with pdfplumber.open("C:\Projects\Staffing\data\CV's\Sara_Vera_Marjanovi_.pdf") as pdf:
    print(pdf.pages[0].extract_text()[:-60])
    #for j, page in enumerate(pdf.pages):
    #    print(page.extract_text())

Sara Vera Marjanović
Data Scientist
Data scientist
Sara is a driven data scientist with a particular specialty in Machine Learning, Deep learning, and Natural Language 
Processing. She has end-to-end experience with wrangling a wide variety of different data sources (from free text to 
biometric data) to create either simple (e.g. clustering, regression) and complex models (e.g. Generative Adversarial 
Models) in the production pipeline. Therefore, she is well-versed in ideation, data collection, data wrangling and 
exploration, as well as model development and trouble-shooting and is comfortable with Python (especially the libraries 
Numpy, Pandas, Tensorflow, Pytorch) as well as SQL, R, and Azure DevOps.
Within her model creation and data exploration, Sara takes special care to limit bias within the dataset and model output, 
and to ensure interpretability of the results. She is diligent and conscientious with her data storytelling and data 
visualization, making her a skilled commun

### Extracting text as string from pdf', save to .csv
Here we read the text from all CV's and save the texts in a .csv file. Each line in the .csv contains name of the colleague, the page number of the CV and the text on that specific page. 

In [ ]:
import csv
import glob
import regex as re
import pdfplumber
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Get list of CV paths
paths = glob.glob("C:\Projects\Staffing\data\CV's\*.pdf")
#print(paths)

stop_words = set(stopwords.words('english'))

with open("cv_test.csv", "w", newline='') as f:
    writer = csv.writer(f)
    csv_input = ["colleague", "total_text", "word_tokens"]
    writer.writerows([csv_input])
    for i, path in enumerate(paths):
        print(f"Currently extracting from {path[31:-4]}'s CV")
        with pdfplumber.open(path) as pdf:
            total_text = ''
            for j, page in enumerate(pdf.pages):

                text = page.extract_text()
                text = re.sub('[^A-Za-z0-9 .,]+', '', text)
                total_text += text
                #word_tokens = word_tokenize(text)
                #filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
                # Only special characters are removed.
                #csv_input = [path[31:-4],j+1, text, filtered_sentence]
                #Write page text to pdf
                #writer.writerows([csv_input])

            word_tokens = word_tokenize(total_text)
            filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
            csv_input = [path[31:-4], total_text, filtered_sentence]
            writer.writerows([csv_input])

Currently extracting from Ahmed_Farooq's CV
Currently extracting from Antonio_Arfe's CV
Currently extracting from Ashish_Meshram's CV
Currently extracting from Ayman_Eleya_Zaki's CV
Currently extracting from Aziz_Ari's CV
Currently extracting from Branislav_Machava's CV
Currently extracting from Daniel_Masson's CV
Currently extracting from Daria_Korzel's CV
Currently extracting from Devichand_Das's CV
Currently extracting from Dinesh_Kumar_Padala's CV
Currently extracting from Ditlev_J_rgensen's CV
Currently extracting from Drisya_Thumba's CV
Currently extracting from Emil_Djursner_Rasmussen's CV
Currently extracting from Eugenia_Borgia's CV
Currently extracting from Hamza_Minhas's CV
Currently extracting from Hatef_Abdollahi's CV
Currently extracting from Jacob_Karlstr_m's CV
Currently extracting from Jan_Neldeberg's CV
Currently extracting from Jayant_Pant's CV
Currently extracting from Jes_Melbye_Chergui's CV
Currently extracting from Julian_Eikhaug's CV
Currently extracting from Kr

: 

### Create TaggedDocument, train Doc2Vec model on data
First we created a TaggedDocument object where we tag each document with the respective name. Each document is a page on the CV.

In [ ]:
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from nltk.tokenize import word_tokenize
import pandas as pd

# load CVs into pandas
df = pd.read_csv("cv_test.csv")

# Tag each CV with name of CV owner
tagged_data = []
tags = df["colleague"].unique().tolist()

# extract all text from a CV and remove stopwords
for tag in tags:
    total_text = df[df["colleague"] == tag]["total_text"]
    word_tokens = word_tokenize(str(total_text))
    filtered_cv = [w for w in word_tokens if not w.lower() in stop_words]
    tagged_data.append(TaggedDocument(words=filtered_cv, tags=[str(tag)]))

print(tagged_data[9])

# model parameters
max_epochs = 200
vec_size = 50
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=5,
                dm =1)

model.build_vocab(tagged_data) # tagged_data

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data, # tagged_data
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v_cvs.model")
print("Model Saved")

TaggedDocument<['9', 'Dinesh', 'Kumar', 'PadalaCV', 'Dinesh', 'Kumar', 'Padala', '2', '...', 'Name', ':', 'total_text', ',', 'dtype', ':', 'object'], ['Dinesh_Kumar_Padala']>
iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
ite

: 

### Use trained model to find best match between a subject and a colleague.
Here we use the trained model to match which CV best fits the given text (subject tested on is Data Migration).

In [ ]:
import regex as re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sys
import os
from scipy import spatial
import numpy

abspath = r"C:\Projects\Staffing\sample_texts"
stop_words = set(stopwords.words('english'))
model= Doc2Vec.load("d2v_cvs.model")
print("Model loaded!")

cvs = pd.read_csv("cv_test.csv")
print("CVs loaded!")
file_name = "data_migration.txt"
print(os.path.join(abspath,file_name))
with open(os.path.join(abspath,file_name)) as f:
    text = f.read().replace('\n', '')
    f.close()

text = re.sub('[^A-Za-z0-9 .,]+', '', text)
text = word_tokenize(text)
filtered_text_object = [w for w in text if not w.lower() in stop_words]
# calculate distances between all cvs and target text
distances = []
for cv in cvs["total_text"]:
    word_tokens = word_tokenize(cv)
    filtered_cv = [w for w in word_tokens if not w.lower() in stop_words]
    distances.append(model.wv.wmdistance(filtered_text_object, filtered_cv))


# print top 5 matches
s = numpy.array(distances)
sort_index = numpy.argsort(s)
cnt = 0
for ind in sort_index:
    if cnt <= 5:
        #print(cvs.iloc[ind]["colleague"])
        print(f"Match Rank: {cnt}, colleague: {cvs.iloc[ind]['colleague']}, distance: {numpy.round(distances[ind],4)}")
        cnt += 1

Model loaded!
CVs loaded!
C:\Projects\Staffing\sample_texts\data_migration.txt
Match Rank: 0, colleague: Mohit_Kumar_Bhati, distance: 0.0253
Match Rank: 1, colleague: Devichand_Das, distance: 0.027
Match Rank: 2, colleague: Zhong_Guan, distance: 0.0273
Match Rank: 3, colleague: Ayman_Eleya_Zaki, distance: 0.0296
Match Rank: 4, colleague: Kristina_Kj_r_Hedegaard, distance: 0.0303
Match Rank: 5, colleague: Branislav_Machava, distance: 0.0314


: 

# subject testet on data science

In [ ]:
import regex as re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sys
import os
from scipy import spatial
import numpy

abspath = r"C:\Projects\Staffing\sample_texts"
stop_words = set(stopwords.words('english'))
model= Doc2Vec.load("d2v_cvs.model")
print("Model loaded!")

cvs = pd.read_csv("cv_test.csv")
print("CVs loaded!")
file_name = "data_science.txt"
print(os.path.join(abspath,file_name))
with open(os.path.join(abspath,file_name)) as f:
    text = f.read().replace('\n', '')
    f.close()

# Testing purposes, to see if distances are correct
# text = cvs.iloc[9]["total_text"]

text = re.sub('[^A-Za-z0-9 .,]+', '', text)
text = word_tokenize(text)
filtered_text_object = [w for w in text if not w.lower() in stop_words]
# calculate distances between all cvs and target text
distances = []
for cv in cvs["total_text"]:
    word_tokens = word_tokenize(cv)
    filtered_cv = [w for w in word_tokens if not w.lower() in stop_words]
    distances.append(model.wv.wmdistance(filtered_text_object, filtered_cv))


# print top 5 matches
s = numpy.array(distances)
sort_index = numpy.argsort(s)
cnt = 0
for ind in sort_index:
    if cnt <= 5:
        #print(cvs.iloc[ind]["colleague"])
        print(f"Match Rank: {cnt}, colleague: {cvs.iloc[ind]['colleague']}, distance: {numpy.round(distances[ind],4)}")
        cnt += 1

Model loaded!
CVs loaded!
C:\Projects\Staffing\sample_texts\data_science.txt
Match Rank: 0, colleague: Ayman_Eleya_Zaki, distance: 0.0205
Match Rank: 1, colleague: Olivia_Essen, distance: 0.025
Match Rank: 2, colleague: Devichand_Das, distance: 0.0252
Match Rank: 3, colleague: Mohit_Kumar_Bhati, distance: 0.026
Match Rank: 4, colleague: Zhong_Guan, distance: 0.0267
Match Rank: 5, colleague: Samay_Mir, distance: 0.0272


: 

In [ ]:
import regex as re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sys
import os
from scipy import spatial
import numpy
model= Doc2Vec.load("d2v_cvs.model")
print("Model loaded!")

cvs = pd.read_csv("cv_test.csv")
print("CVs loaded!")
abspath = r"C:\Projects\Staffing\sample_texts"
stop_words = set(stopwords.words('english'))

file_name = "software_development_python.txt"
print(os.path.join(abspath,file_name))
with open(os.path.join(abspath,file_name)) as f:
    text = f.read().replace('\n', '')
    f.close()

text = re.sub('[^A-Za-z0-9 .,]+', '', text)
text = word_tokenize(text)
filtered_text_object = [w for w in text if not w.lower() in stop_words]
# calculate distances between all cvs and target text
distances = []
for cv in cvs["total_text"]:
    word_tokens = word_tokenize(cv)
    filtered_cv = [w for w in word_tokens if not w.lower() in stop_words]
    distances.append(model.wv.wmdistance(filtered_text_object, filtered_cv))


# print top 5 matches
s = numpy.array(distances)
sort_index = numpy.argsort(s)
cnt = 0
for ind in sort_index:
    if cnt <= 5:
        #print(cvs.iloc[ind]["colleague"])
        print(f"Match Rank: {cnt}, colleague: {cvs.iloc[ind]['colleague']}, distance: {numpy.round(distances[ind],4)}")
        cnt += 1

Model loaded!
CVs loaded!
C:\Projects\Staffing\sample_texts\software_development_python.txt
Match Rank: 0, colleague: Shivam_Singh, distance: 0.0425
Match Rank: 1, colleague: Jacob_Karlstr_m, distance: 0.0619
Match Rank: 2, colleague: Daniel_Masson, distance: 0.063
Match Rank: 3, colleague: Rajitha_Dodda, distance: 0.0699
Match Rank: 4, colleague: Sara_Vera_Marjanovi_, distance: 0.073
Match Rank: 5, colleague: Hatef_Abdollahi, distance: 0.0741


: 

In [ ]:
import regex as re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sys
import os
from scipy import spatial
import numpy

abspath = r"C:\Projects\Staffing\sample_texts"
stop_words = set(stopwords.words('english'))
model= Doc2Vec.load("d2v_cvs.model")
print("Model loaded!")

cvs = pd.read_csv("cv_test.csv")
print("CVs loaded!")
file_name = "fraud_detection.txt"
print(os.path.join(abspath,file_name))
with open(os.path.join(abspath,file_name)) as f:
    text = f.read().replace('\n', '')
    f.close()

# Testing purposes, to see if distances are correct
# text = cvs.iloc[9]["total_text"]

text = re.sub('[^A-Za-z0-9 .,]+', '', text)
text = word_tokenize(text)
filtered_text_object = [w for w in text if not w.lower() in stop_words]
# calculate distances between all cvs and target text
distances = []
for cv in cvs["total_text"]:
    word_tokens = word_tokenize(cv)
    filtered_cv = [w for w in word_tokens if not w.lower() in stop_words]
    distances.append(model.wv.wmdistance(filtered_text_object, filtered_cv))


# print top 5 matches
s = numpy.array(distances)
sort_index = numpy.argsort(s)
cnt = 0
for ind in sort_index:
    if cnt <= 5:
        #print(cvs.iloc[ind]["colleague"])
        print(f"Match Rank: {cnt}, colleague: {cvs.iloc[ind]['colleague']}, distance: {numpy.round(distances[ind],4)}")
        cnt += 1

Model loaded!
CVs loaded!
C:\Projects\Staffing\sample_texts\fraud_detection.txt
Match Rank: 0, colleague: Shivam_Singh, distance: 0.0236
Match Rank: 1, colleague: Jacob_Karlstr_m, distance: 0.0389
Match Rank: 2, colleague: Daniel_Masson, distance: 0.0398
Match Rank: 3, colleague: Hatef_Abdollahi, distance: 0.0459
Match Rank: 4, colleague: Sara_Vera_Marjanovi_, distance: 0.0485
Match Rank: 5, colleague: Ayman_Eleya_Zaki, distance: 0.0525


: 